In [1]:
%run seq2seq.ipynb

In [2]:
class Attention(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.bilinear = nn.Bilinear(dim, dim, 1)
        
    def forward(self, source_h, target_h):
        '''
        # TODO: repeat copies data, use expand
        
        source_h => (num_sentences * num_words * embedding_dim)
        target_h => (num_sentences * embedding_dim) 
        '''
        num_sentences, num_words = source_h.shape[:-1]
        target_h = target_h.repeat(1, num_words).reshape(num_words * num_sentences, -1)
        raw_weights = self.bilinear(target_h, source_h.reshape(num_words * num_sentences, 
                                                               -1)).reshape(num_sentences, num_words)
        weights = torch.softmax(f.relu(raw_weights), dim=1)
        
        return torch.sum(weights.unsqueeze(-1) * source_h, dim=1)

In [3]:
source_h = torch.Tensor(np.random.randint(0, 10, (2, 4, 10)))
target_h = torch.Tensor(np.random.randint(0, 10, (2, 10)))
a = Attention(10)
a(source_h, target_h)

tensor([[5.1823, 8.9290, 2.2333, 1.0917, 0.2633, 2.2023, 1.0913, 5.9290, 4.0307,
         8.8173],
        [6.6327, 1.2118, 4.9474, 2.3144, 8.5292, 1.8956, 0.1587, 5.9996, 3.1568,
         0.3140]], grad_fn=<SumBackward2>)

In [26]:
torch.cuda.set_device(0)

In [129]:
class DecoderAttention(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_dim, num_layers, padding_dim=0, start_dim=1):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim, padding_dim)
        self.decoder = nn.LSTM(embedding_dim + hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.word_predictor = nn.Linear(hidden_dim, num_embeddings)
        self.start_dim = start_dim
        
        self.attention = Attention(hidden_dim)
        
    def forward(self, output, h, c, input=None, max_sen_len=20):
        
        source_h = output
        num_sentences, num_words, hidden_dim = output.shape
        words_selected = torch.cuda.LongTensor([[self.start_dim] for _ in range(num_sentences)])

        decoder_context = torch.zeros(num_sentences, hidden_dim).cuda()
        pred = []
        
        for i in range(max_sen_len):
            embeddings = self.embedding(words_selected)
            decoder_input = torch.cat((embeddings, decoder_context.unsqueeze(1)), dim=-1)
            y_t, (h, c) = self.decoder(decoder_input, (h, c))
            orig_shape = y_t.shape
            pred_t = self.word_predictor(y_t.reshape(orig_shape[0]*orig_shape[1], 
                                                     -1)).reshape(*orig_shape[:-1], -1).squeeze(1)
            pred.append(pred_t)
            words_selected = torch.max(pred_t, dim=1)[1].unsqueeze(1)
            decoder_context = self.attention(source_h, h[-1])

        pred = torch.stack(pred, dim=1)
        return torch.log_softmax(pred, dim=-1)   

In [134]:
class Seq2SeqWithAttention(nn.Module):
    def __init__(self, source_vocab_len, target_vocab_len, embedding_dim, hidden_dim,
                       start_dim=1, padding_dim=0, num_layers=1):
        super().__init__()
        self.encoder = Encoder(source_vocab_len, embedding_dim, hidden_dim, num_layers, padding_dim)
        self.decoder = DecoderAttention(target_vocab_len, embedding_dim, hidden_dim, num_layers, padding_dim, start_dim)
        
    def forward(self, source_input, target_input=None):
        output, h, c = self.encoder(source_input)
        return self.decoder(output, h, c, target_input)

`s = Seq2SeqWithAttention(10, 12, 9, 8)`

`s.cuda()`

`input = torch.cuda.LongTensor(np.random.randint(0, 10, (3, 6)))`

`s(input)`